<a href="https://colab.research.google.com/github/Mhmdaris15/Face-eye-detection/blob/main/Face_%26_Eye_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Face & Eye Detection using HAAR Cascade Classifiers

In [ ]:
from google.colab import files as colabfiles
from google.colab import drive
from google.colab.patches import cv2_imshow

In [ ]:
filess = colabfiles.upload()

Saving Elon_Musk_2015.jpg to Elon_Musk_2015.jpg


In [ ]:
import numpy as np
import cv2

# We point OpenCV's CascadeClassifier function to where our 
# classifier (XML file format) is stored
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load our image, resize and then convert it to grayscale
image = cv2.imread('Elon_Musk_2015.jpg')
scale_percent = 20 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
image = cv2.resize(image, dsize=dim, fx=.5, fy=.5, interpolation=cv2.INTER_LANCZOS4)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Our classifier returns the ROI of the detected face as a tuple
# It stores the top left coordinate and the bottom right coordiantes
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is None:
    print("No faces found")

# We iterate through our faces array and draw a rectangle
# over each face in faces
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (127,0,255), 2)
    cv2_imshow(image)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

### Let's combine face and eye detection

In [ ]:
import numpy as np
import cv2
 
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')
 
img = cv2.imread('Elon_Musk_2015.jpg')
scale_percent = 20 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
img = cv2.resize(img, dsize=dim, fx=.5, fy=.5, interpolation=cv2.INTER_LANCZOS4)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(gray, 1.5, 6)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No Face Found")
amountLoop = 0
for (x,y,w,h) in faces:
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(127,0,255),2)
    cv2_imshow(img)
    cv2.waitKey(0)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_classifier.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        if amountLoop == 2 :
          break
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
        cv2_imshow(img)
        cv2.waitKey(0)
        amountLoop += 1
    
    
cv2.destroyAllWindows()

### Let's make a live face & eye detection, keeping the face inview at all times

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img
    
    for (x,y,w,h) in faces:
        x = x - 50
        w = w + 50
        y = y - 50
        h = h + 50
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_classifier.detectMultiScale(roi_gray)
        
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2) 
            
    roi_color = cv2.flip(roi_color,1)
    return roi_color

# while True:
#   filename = take_photo()
#   # cv2_imshow(face_detector('photo.jpg'))
#   img = cv2.imread('photo.jpg')
#   cv2_imshow(face_detector(img))
#   if cv2.waitKey(1) == 13 :
#     break
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    cv2_imshow(face_detector(frame))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      

In [ ]:
import cv2
import numpy as np

face_clf = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_clf = cv2.CascadeClassifier('haarcascade_eye.xml')

def face_detector(image, size=1.5):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_corners = face_clf.detectMultiScale(gray, 1.4, 3)
    if face_corners is None:
        return image
    for (x,y,w,h) in face_corners:
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,254), 2)
        roi_color = image[y:y+h, x:x+w]
        roi_gray = gray[y:y+h, x:x+w]
        eye_corners = eye_clf.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eye_corners:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255,0,0), 2)
            #cv2.circle(roi_color, (int((ex+ew)/2), int((ey+eh)/2)), 0, (255,0,0), 2)
    roi_color = cv2.flip(roi_color, 1)
    return roi_color
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    cv2_imshow(face_detector(frame))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      

### Tuning Cascade Classifiers

*ourClassifier*.**detectMultiScale**(input image, **Scale Factor** , **Min Neighbors**)

- **Scale Factor**
Specifies how much we reduce the image size each time we scale. E.g. in face detection we typically use 1.3. This means we reduce the image by 30% each time it’s scaled. Smaller values, like 1.05 will take longer to compute, but will increase the rate of detection.



- **Min Neighbors**
Specifies the number of neighbors each potential window should have in order to consider it a positive detection. Typically set between 3-6. 
It acts as sensitivity setting, low values will sometimes detect multiples faces over a single face. High values will ensure less false positives, but you may miss some faces.  


In [ ]:
import cv2
import numpy as np
# cv2.namedWindow('Aris Resized', cv2.WINDOW_NORMAL)
image = cv2.imread('../../../../../Downloads/files/aris-san.jpg', -1)
imageS = cv2.resize(image, (int(image.shape[1]/2), int(image.shape[0]/2)))
imageS = imageS[150:700, 175:700]
gray = cv2.cvtColor(imageS, cv2.COLOR_BGR2GRAY)
print(imageS.shape)
cv2.imshow('Aris Grayed', gray)
cv2.imshow('Aris Handsome', imageS)
cv2.waitKey(0)
cv2.destroyAllWindows()

face_detector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_detector = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')

print(face_detector)

face_corner = face_detector.detectMultiScale(gray, 1.4, 5)

if face_corner is None:
    print('No result found here')

for (x,y,w,h) in face_corner:
    cv2.rectangle(imageS, (x,y), (x+w, y+h), (0,0,255), 2)
    cv2.imshow('Detected', imageS)
    cv2.waitKey(0)
    RoiGray = gray[y:y+h, x:x+w]
    RoiColor = imageS[y:y+h, x:x+w]
    eye_corner = eye_detector.detectMultiScale(RoiGray, 1.3, 5)
    for (ex, ey, ew, eh) in eye_corner:
        cv2.rectangle(RoiColor, (ex, ey), (ex+ew, ey+eh), (255,0,0), 3)
        cv2.imshow('Final Image', imageS)
        cv2.waitKey(0)
cv2.destroyAllWindows()

(550, 525, 3)
<CascadeClassifier 0000008D609905D0>
